In [20]:
import pandas as pd
import numpy as np
import gensim

In [74]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [75]:
data['tweet'][0]

'#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone'

In [76]:
import PreProcessing as pp

In [78]:
data['tweet'] = pp.corpus_pre_processor(data['tweet'])
test['tweet'] = pp.corpus_pre_processor(test['tweet'])

100%|██████████████████████████████████████████████████████████████████████████████| 1953/1953 [00:27<00:00, 70.55it/s]


In [85]:
dataW2V['tweet'] = pd.concat((data['tweet'],test['tweet']), axis=0)

In [86]:
data.shape, test.shape, dataW2V.shape

((7920, 3), (1953, 2), (9874,))

In [89]:
dataW2V.to_frame()

,tweet
0,fingerprint pregnancy test httpsgoo glhmfqv an...
1,finally transparant silicon case thank uncle y...
2,love talk makememorie unplug relax iphone smar...
3,wired know george way iphone cute daventry hom...
4,amazing service apple talk question pay stupid...
...,...
1949,available hoodie check httpzetasupplies co ukp...
1950,crack right screen actually provide durable sc...
1951,codeofinter adobe big time include apple
1952,finally thanx father samsung galaxy s gift fat...


In [10]:
data['label'].unique()

array([0, 1], dtype=int64)

In [83]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [90]:
%%time
# build word2vec model
w2v_num_features = 300
w2v_model = gensim.models.Word2Vec(dataW2V['tweet'], size=w2v_num_features, window=150,
                                   min_count=10, workers=4, iter=5)    

2020-12-14 12:51:32,487 : INFO : collecting all words and their counts
2020-12-14 12:51:32,488 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2020-12-14 12:51:32,489 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-12-14 12:51:32,556 : INFO : collected 27 word types from a corpus of 897330 raw words and 9873 sentences
2020-12-14 12:51:32,556 : INFO : Loading a fresh vocabulary
2020-12-14 12:51:32,557 : INFO : effective_min_count=10 retains 27 unique words (100% of original 27, drops 0)
2020-12-14 12:51:32,557 : INFO : effective_min_count=10 leaves 897330 word corpus (100% of original 897330, drops 0)
2020-12-14 12:51:32,558 : INFO : deleting the raw counts dictionary of 27 items
2020-12-14 12:51:32,558 : INFO : sample=0.001 downsamples 27 most-common words
2020-12-14 12:51:32,558 : INFO : downsampling leaves estimated 160010 word corpus (17.8% of prior 897330)
2020-12-

Wall time: 1.77 s


In [91]:
w2v_model

In [92]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [93]:
# generate averaged word vector features from word2vec model
avg_wv_train_features = averaged_word2vec_vectorizer(corpus=data['tweet'], model=w2v_model,
                                                     num_features=w2v_num_features)

In [94]:
print('Word2Vec model:> Train features shape:', avg_wv_train_features.shape)

Word2Vec model:> Train features shape: (7920, 300)


In [95]:
avg_wv_train_features

array([[-0.0135758 , -0.01645124,  0.00468796, ...,  0.04711473,
        -0.03116195,  0.0812814 ],
       [-0.01671251, -0.01801623,  0.00775362, ...,  0.01715923,
        -0.01782746,  0.01168521],
       [ 0.10088126, -0.03124108, -0.07848133, ...,  0.0322214 ,
        -0.00849604,  0.03712542],
       ...,
       [ 0.07079149, -0.02354569, -0.04577431, ...,  0.05033043,
        -0.02642762,  0.08659086],
       [ 0.03954863, -0.03027592, -0.0319475 , ...,  0.02359941,
        -0.02160646,  0.02459969],
       [ 0.01716689, -0.00352843, -0.01789289, ...,  0.05326048,
        -0.01454194,  0.08386773]])

In [96]:
data.shape

(7920, 3)

In [97]:
train = pd.concat((data['label'],pd.DataFrame(avg_wv_train_features)),axis = 1)

In [98]:
train.shape

(7920, 301)

In [99]:
train.head()

,label,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,0,-0.013576,-0.016451,0.004688,-0.035999,0.000040,-0.013737,0.007057,-0.042352,-0.038366,...,-0.000739,-0.006704,-0.020937,-0.009889,-0.015489,-0.007260,-0.004652,0.047115,-0.031162,0.081281
1,0,-0.016713,-0.018016,0.007754,-0.046623,0.010912,-0.041410,-0.015443,-0.058151,-0.008805,...,0.005780,-0.011579,-0.000208,-0.015387,-0.009484,-0.006502,-0.033964,0.017159,-0.017827,0.011685
2,0,0.100881,-0.031241,-0.078481,-0.051787,-0.003125,-0.039954,-0.006787,-0.048855,-0.017163,...,0.010730,-0.009638,-0.013836,-0.014167,-0.007706,-0.005716,-0.014925,0.032221,-0.008496,0.037125
3,0,0.063640,-0.023770,-0.056930,-0.040672,-0.002884,-0.037690,-0.015767,-0.049543,-0.001706,...,0.004242,-0.004762,-0.016604,-0.009860,-0.016649,-0.006179,-0.010324,0.040459,-0.025028,0.065541
4,1,0.068801,-0.017947,-0.079257,-0.018589,0.000548,-0.065316,-0.038746,-0.048102,0.053128,...,0.016212,-0.016163,0.007119,-0.011498,0.001929,-0.004101,-0.042474,0.003034,-0.004947,-0.004167


In [100]:
y = train['label']

In [101]:
y

0       0
1       0
2       0
3       0
4       1
       ..
7915    0
7916    0
7917    0
7918    0
7919    0
Name: label, Length: 7920, dtype: int64

In [102]:
train_new = train.drop(columns = ['label'])

In [103]:
%%time

# Logistic Regression model on BOW features
from sklearn.linear_model import LogisticRegression

# instantiate model
lr = LogisticRegression(penalty='l2', max_iter=500, C=1, solver='lbfgs', random_state=42)

# train model
lr.fit(train_new, y)

# predict on test data
#lr_bow_predictions = lr.predict(train_new)

Wall time: 180 ms


LogisticRegression(C=1, max_iter=500, random_state=42)

In [104]:
test = pd.read_csv("test.csv")

In [106]:
test['tweet'] = pp.corpus_pre_processor(test['tweet'])

100%|██████████████████████████████████████████████████████████████████████████████| 1953/1953 [00:36<00:00, 53.40it/s]


In [107]:
# generate averaged word vector features from word2vec model
avg_wv_test_features = averaged_word2vec_vectorizer(corpus=test['tweet'], model=w2v_model,
                                                     num_features=w2v_num_features)

In [108]:
test_df = pd.DataFrame(avg_wv_test_features)

In [109]:
test_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.070936,-0.023778,-0.049989,-0.043263,-0.000850,-0.007813,0.013064,-0.030286,-0.029372,-0.021425,...,0.020127,-0.004029,-0.022633,-0.014148,-0.013533,-0.010730,-0.001039,0.048662,-0.015966,0.080909
1,0.069721,-0.031457,-0.065354,-0.047420,0.001852,-0.043339,-0.020405,-0.056624,0.010008,-0.005564,...,-0.003726,-0.013186,0.006407,-0.012018,-0.005639,-0.001618,-0.033842,0.010550,-0.014104,-0.002666
2,0.131577,-0.010762,-0.112197,-0.022459,-0.007883,-0.034832,-0.017611,-0.033606,0.020690,-0.003339,...,0.025508,-0.010628,-0.005691,-0.012008,-0.004645,-0.004579,-0.029457,0.020534,-0.003398,0.031697
3,0.046355,-0.012002,-0.039004,-0.034142,-0.005352,-0.033072,-0.003550,-0.040762,0.000557,-0.013735,...,0.027908,-0.004209,-0.022418,-0.014786,-0.014737,-0.011680,-0.004135,0.056277,-0.021434,0.100527
4,0.050731,-0.039912,-0.049462,-0.056823,0.002062,-0.049470,-0.017850,-0.065559,0.009643,-0.010903,...,-0.017846,-0.012006,-0.006182,-0.010086,-0.014012,-0.000567,-0.026907,0.020025,-0.024220,0.013697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,0.086568,-0.040283,-0.057642,-0.067015,0.005959,-0.025618,-0.003938,-0.049605,-0.026344,-0.018686,...,-0.008656,-0.010916,0.007811,-0.014370,-0.004339,-0.006945,-0.034632,-0.002840,-0.003371,-0.036653
1949,0.149855,-0.037981,-0.120456,-0.043776,-0.002163,-0.075781,-0.041393,-0.053914,0.057340,0.014037,...,0.010334,-0.021271,0.005422,-0.014836,0.009080,-0.004931,-0.041168,-0.009220,0.014358,-0.035421
1950,0.143842,-0.024531,-0.116756,-0.035910,-0.001105,-0.029470,-0.015676,-0.034667,-0.005762,-0.013503,...,0.025445,-0.013822,-0.000671,-0.015543,0.001631,-0.007908,-0.030910,0.017247,-0.001947,0.021766
1951,0.121918,-0.026666,-0.089852,-0.037917,-0.007479,-0.027807,-0.004842,-0.031255,-0.039905,-0.036717,...,0.031391,-0.004006,-0.010918,-0.016361,-0.010537,-0.005085,-0.009485,0.037385,-0.008667,0.062723


In [110]:
# predict on test data
lr_predictions = lr.predict(test_df)

In [111]:
lr_predictions

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [112]:
submit = pd.read_csv("sample_submission.csv")

In [113]:
submit['label'] = lr_predictions

In [114]:
submit

,id,label
0,7921,1
1,7922,0
2,7923,1
3,7924,1
4,7925,0
...,...,...
1948,9869,0
1949,9870,0
1950,9871,1
1951,9872,1


In [115]:
import time
timestr = time.strftime("%Y%m%d%H%M%S")
print(timestr)

20201214125354


In [116]:
import time
timestr = time.strftime("%Y%m%d%H%M%S")
filename = "sample_submission_" + timestr + ".csv"


In [117]:
submit.to_csv(filename)